# Problem
Construct the Boolean formula to model the following problem:

> Given a 3 × 3 chessboard, place three Rook pieces such that there are no two pieces that can attack each other. A Rook piece can move either vertically or horizontally.

Determine whether the formula is satisfiable or not. Explain your answer.


<div align="center">

| $a_{11}$ | $a_{12}$ | $a_{13}$ |
|----------|----------|----------|
| $a_{21}$ | $a_{22}$ | $a_{23}$ |
| $a_{31}$ | $a_{32}$ | $a_{33}$ |

</div>

**Instructions:**

* Define nine Boolean variables $a_{ij}$ where $i, j ∈ {1,2,3}$. The variable $a_{ij}$ is active (without negation) if a Rook piece is placed in the $i$‑row and $j$‑th column; otherwise it is passive (with negation).
* You may need to define the formula as

  $$
  F = F₁₁ ∧ F₁₂ ∧ F₁₃ ∧ F₂₁ ∧ F₂₂ ∧ F₂₃ ∧ F₃₁ ∧ F₃₂ ∧ F₃₃ ∧ R₁ ∧ R₂ ∧ R₃ ∧ C₁ ∧ C₂ ∧ C₃
  $$

  where $Fᵢⱼ$ is a subformula when assuming a Rook piece is placed in the $i$‑row and $j$‑th column, $Rᵢ$ (respectively $Cⱼ$) is a subformula representing the condition that at least one Rook piece is placed in the $i$‑row (respectively $j$‑column).
* You may also need to use implication formula to define $Fᵢⱼ$: if a Rook piece is placed in this cell then one cannot place Rook in “these” cells.


# Solution

First we define nine Boolean variables $a_{ij}$ where $i, j ∈ {1,2,3}$

In [86]:
# import z3
from z3 import *

#define the variables
a = [[Bool(f'a_{i}{j}') for j in range(1, 4)] for i in range(1, 4)]
print(a)

[[a_11, a_12, a_13], [a_21, a_22, a_23], [a_31, a_32, a_33]]


 Next we define the condition $R_i$ and $C_i$ where $i\in{1,2,3}$ there at least one rook placed at each $i$-row and $i$-column. The formula can represented by
\begin{align*}
  R_i &:= a_{i1} \vee a_{i2} \vee a_{i3}\\
  C_i &:= a_{1i} \vee a_{2i} \vee a_{3i}
\end{align*}

In [87]:
R = [Or(a[i][0], a[i][1], a[i][2]) for i in range(3)]
C = [Or(a[0][i], a[1][i], a[2][i]) for i in range(3)]

print(f'Row: {R}')
print(f'Column: {C}')

Row: [Or(a_11, a_12, a_13), Or(a_21, a_22, a_23), Or(a_31, a_32, a_33)]
Column: [Or(a_11, a_21, a_31), Or(a_12, a_22, a_32), Or(a_13, a_23, a_33)]


Then the $F_{ij}$ formula can represented by
$$F_{ij}:=a_{ij}\implies\left(\bigwedge_{\substack{k=1\\k\neq i}}^{3}\neg a_{kj}
\land
\bigwedge_{\substack{\ell=1\\\ell\neq j}}^{3}\neg a_{i\ell}\right)
$$

In [88]:
F=[]
for i in range(3):
    row = []
    for j in range(3):
        not_same_col = [Not(a[k][j]) for k in range(3) if k != i]
        not_same_row = [Not(a[i][l]) for l in range(3) if l != j]
        constraint = Implies(a[i][j], And(*(not_same_col + not_same_row)))
        row.append(constraint)
    F.append(row)
        
print(f'F: {F}')

F: [[Implies(a_11,
        And(Not(a_21), Not(a_31), Not(a_12), Not(a_13))), Implies(a_12,
        And(Not(a_22), Not(a_32), Not(a_11), Not(a_13))), Implies(a_13,
        And(Not(a_23), Not(a_33), Not(a_11), Not(a_12)))], [Implies(a_21,
        And(Not(a_11), Not(a_31), Not(a_22), Not(a_23))), Implies(a_22,
        And(Not(a_12), Not(a_32), Not(a_21), Not(a_23))), Implies(a_23,
        And(Not(a_13), Not(a_33), Not(a_21), Not(a_22)))], [Implies(a_31,
        And(Not(a_11), Not(a_21), Not(a_32), Not(a_33))), Implies(a_32,
        And(Not(a_12), Not(a_22), Not(a_31), Not(a_33))), Implies(a_33,
        And(Not(a_13), Not(a_23), Not(a_31), Not(a_32)))]]


The last step is combine that all formula with conjunction, such that
$$ F = F_{11} \land F_{12} \land F_{13} \land F_{21} \land F_{22} \land F_{23} \land F_{31} \land F_{32} \land F_{33} \land R_1 \land R_2 \land R_3 \land C_1 \land C_2 \land C_3$$

and solve it!

In [89]:
formula = And(*R,*C,*[F[i][j] for i in range(3) for j in range(3)])

s = Solver()
s.add(formula)

s.check()

sat

And yes... the model is satisfiable. The model can we check by:

In [90]:
s.model()

[a_11 = False,
 a_22 = False,
 a_21 = True,
 a_23 = False,
 a_32 = False,
 a_12 = True,
 a_13 = False,
 a_33 = True,
 a_31 = False]

The Ilustration of example model can be draw like this
<div align="center">

|  | 🏛️ |  |
|----------|----------|----------|
| 🏛️ |  |  |
|  |  | 🏛️ |

</div>

Now if we want to check all models such that the formula is satisfiable, consider the fact of combination of the case we want can be calculate by permutation of multiset
$$\text{Total Models }= \frac{(3!)^2}{3!}=3!=6$$

In [91]:
# List to store all models
models = []
count = 1

while s.check() == sat:
    m = s.model()
    models.append(m)
    # Print the model
    print(f"Model {count}:")
    print({v: m[v] for v in a[0] + a[1] + a[2]})
    count += 1

    # Add blocking clause to exclude current model
    block = [v == (not is_true(m[v])) for v in a[0] + a[1] + a[2]]
    s.add(Or(block))

print(f"\nTotal solutions found: {len(models)}")

Model 1:
{a_11: False, a_12: True, a_13: False, a_21: True, a_22: False, a_23: False, a_31: False, a_32: False, a_33: True}
Model 2:
{a_11: False, a_12: False, a_13: True, a_21: False, a_22: True, a_23: False, a_31: True, a_32: False, a_33: False}
Model 3:
{a_11: True, a_12: False, a_13: False, a_21: False, a_22: False, a_23: True, a_31: False, a_32: True, a_33: False}
Model 4:
{a_11: True, a_12: False, a_13: False, a_21: False, a_22: True, a_23: False, a_31: False, a_32: False, a_33: True}
Model 5:
{a_11: False, a_12: True, a_13: False, a_21: False, a_22: False, a_23: True, a_31: True, a_32: False, a_33: False}
Model 6:
{a_11: False, a_12: False, a_13: True, a_21: True, a_22: False, a_23: False, a_31: False, a_32: True, a_33: False}

Total solutions found: 6


The Illustration of all solutions
<div style="display: flex; justify-content: center;">
  <div>
    <table border="1">
      <tr><td> </td><td>🏛️</td><td> </td></tr>
      <tr><td>🏛️</td><td> </td><td> </td></tr>
      <tr><td> </td><td> </td><td>🏛️</td></tr>
    </table>
  </div>

  <div>
    <table border="1">
      <tr><td> </td><td> </td><td>🏛️</td></tr>
      <tr><td> </td><td>🏛️</td><td> </td></tr>
      <tr><td>🏛️</td><td> </td><td> </td></tr>
    </table>
  </div>

  <div>
    <table border="1">
      <tr><td>🏛️</td><td> </td><td></td></tr>
      <tr><td> </td><td> </td><td>🏛️</td></tr>
      <tr><td> </td><td>🏛️</td><td> </td></tr>
    </table>
  </div>
</div>

$$
\,
$$

<div style="display: flex; justify-content: center;">
  <div>
    <table border="1">
      <tr><td>🏛️</td><td> </td><td> </td></tr>
      <tr><td> </td><td>🏛️</td><td> </td></tr>
      <tr><td> </td><td> </td><td>🏛️</td></tr>
    </table>
  </div>

  <div>
    <table border="1">
      <tr><td> </td><td>🏛️</td><td></td></tr>
      <tr><td> </td><td></td><td>🏛️</td></tr>
      <tr><td>🏛️</td><td> </td><td> </td></tr>
    </table>
  </div>

  <div>
    <table border="1">
      <tr><td></td><td> </td><td>🏛️</td></tr>
      <tr><td>🏛️</td><td> </td><td></td></tr>
      <tr><td> </td><td>🏛️</td><td> </td></tr>
    </table>
  </div>
</div>